<a href="https://colab.research.google.com/github/Danh1905/Scalable-Project/blob/main/scalableProj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To upload a file from your local system, you can use `google.colab.files.upload()`:

In [37]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')

Saving Customer-Churn-Prediction.csv to Customer-Churn-Prediction.csv
User uploaded file "Customer-Churn-Prediction.csv" with length 977501 bytes


Install pyspark


In [38]:
!pip install pyspark py4j

In [83]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Read CSV").getOrCreate()
churn_rate = spark.read.csv("/content/Customer-Churn-Prediction.csv", header=True, inferSchema=True)
churn_rate.printSchema()


root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: string (nullable = true)
 |-- Churn: string (nullable = true)



**Preprocessing**

In [84]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import col, sum, when, trim, lit

#Drop the useless ID
churn_rate = churn_rate.drop("customerID")

# Fix TotalCharges (Convert String to Number)
# Replace empty strings with nulls first, then cast to Double, and fill actual nulls with 0
churn_rate = churn_rate.withColumn("TotalCharges",
                                   when(trim(col("TotalCharges")) == "", lit(None)).otherwise(col("TotalCharges")))
churn_rate = churn_rate.withColumn("TotalCharges", col("TotalCharges").cast(DoubleType()))
churn_rate = churn_rate.fillna(0, subset=["TotalCharges"])

# Check null values
def count_nulls(churn_rate):
    null_counts = churn_rate.select([
        sum(when(col(c).isNull(), 1).otherwise(0)).alias(c)
        for c in churn_rate.columns
    ])
    return null_counts

null_churn_rate = count_nulls(churn_rate)
null_churn_rate.show()

+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|Contract|PaperlessBilling|PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|     0|            0|      0|         0|     0|           0|            0|              0|             0|           0|               0|          0|          0|              0|       0|               0|            0|             0|           0|   

In [103]:
# ENCODE ALL TEXT COLUMNS

# List of columns that are text (categorical) and need to be numbers
# Note: I excluded 'Churn' because we handle it separately as the label
categorical_cols = [
    "gender", "Partner", "Dependents", "PhoneService", "MultipleLines",
    "InternetService", "OnlineSecurity", "OnlineBackup", "DeviceProtection",
    "TechSupport", "StreamingTV", "StreamingMovies", "Contract",
    "PaperlessBilling", "PaymentMethod"
]

# This loop converts each text column into a number column
# Example: "gender" becomes "gender_index" (0 or 1)
indexers = []
for column in categorical_cols:
    indexer = StringIndexer(inputCol=column, outputCol=column + "_index")
    churn_rate = indexer.fit(churn_rate).transform(churn_rate)

# Don't forget to encode the Target variable (Churn) too!
label_indexer = StringIndexer(inputCol="Churn", outputCol="label")
churn_rate = label_indexer.fit(churn_rate).transform(churn_rate)



In [104]:
# ASSEMBLE

# Gather all the NEW numbered columns + the original numerical columns
input_features = [c + "_index" for c in categorical_cols] + \
                 ["SeniorCitizen", "tenure", "MonthlyCharges", "TotalCharges"]

# Squash them into one vector
assembler = VectorAssembler(inputCols=input_features, outputCol="features")
final_data = assembler.transform(churn_rate)

# Show the result
final_data.select("features", "label").show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(19,[0,1,3,4,5,7,...|  0.0|
|(19,[5,6,8,12,13,...|  0.0|
|(19,[5,6,7,14,16,...|  1.0|
|[0.0,0.0,0.0,1.0,...|  0.0|
|(19,[0,16,17,18],...|  1.0|
+--------------------+-----+
only showing top 5 rows


In [105]:
from pyspark.ml.feature import StandardScaler

# SCALE THE FEATURES

scaler = StandardScaler(inputCol="features", outputCol="scaled_features",
                        withStd=True, withMean=False)

# Compute the scaling statistics
scalerModel = scaler.fit(final_data)

# Apply the scaling
data_ready = scalerModel.transform(final_data)

# Show the difference
data_ready.select("features", "scaled_features").show(5, truncate=False)

+-----------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                           |scaled_features                                                                                                                                                                                                                                                |
+-----------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------